<a href="https://colab.research.google.com/github/aswindcu2025/churn_prediction_telecom_ds/blob/areen/Model_building_%26_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
df_train=pd.read_csv("/content/Training_data.csv")
df_test=pd.read_csv("/content/Test_data.csv")

In [3]:
df_train.head()

,tenure_scaled,MonthlyCharges_scaled,TotalCharges_scaled,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,-0.463037,0.002935,-0.416007,0,0,0,1,1,0,1,1,0,1,0,0,1,1,0,3,0
1,0.880735,1.078118,1.257246,0,0,0,0,1,1,2,0,1,0,0,1,1,2,1,0,0
2,-1.277445,-1.373033,-0.995434,1,0,1,0,0,2,1,0,0,0,0,0,0,0,1,2,1
3,-1.155283,0.180747,-0.900800,1,0,0,0,1,0,2,0,0,0,0,0,0,0,1,2,1
4,-1.318165,-0.095111,-1.005780,1,0,0,1,1,1,1,1,1,0,1,0,0,2,1,0,0


In [8]:
X = df_train.drop('Churn', axis=1)
y = df_train['Churn']

In [9]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=.2,random_state=42)

In [33]:
model_params= {
    'lr': {
        'model': LogisticRegression(),
        'params':{
            'C': [1,5,10]
        }
    },
    'DTC': {
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion': ["gini", "entropy"],
            'max_depth' : [2, 3, 5, 10, 20, None],
            'min_samples_split' : [2, 5, 10, 20, 50],
            'splitter' : ["best", "random"]
          }
    },
    'rfc':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10,15,20],
            'max_depth' : [None, 10, 20, 30, 50],
            'max_features' :['sqrt', 'log2', None]

        }
    },
    'xgb':{
        'model':XGBClassifier(),
        'params':{
            'learning_rate':[0.01,0.1,0.2],
            'n_estimators':[100,200,300],
            'max_depth':[3,5,7],
            'subsample':[0.7,1.0],
            'colsample_bytree':[0.7,1.0]
}

    },
    'bagging':{
        'model': BaggingClassifier(),
        'params': {
            'estimator': [DecisionTreeClassifier(),DecisionTreeClassifier(max_depth=5),DecisionTreeClassifier(max_depth=10)],
            'n_estimators':[50,100,150,200],
            'max_samples':[0.25,0.5, 0.7, 0.8],
            'max_features' : [0.5, 0.7, 1.0]



        }
    }
}

In [34]:
from sklearn import model_selection
score=[ ]

for model_name, mp in model_params.items():
  gscv=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)

  gscv.fit(xtrain,ytrain)

  score.append({'model':model_name,
                'best_score': gscv.best_score_,
                'best_params':gscv.best_params_})

In [35]:
pd.DataFrame(score)

,model,best_score,best_params
0,lr,0.775227,{'C': 5}
1,DTC,0.782628,"{'criterion': 'gini', 'max_depth': None, 'min_..."
2,rfc,0.825982,"{'max_depth': 50, 'max_features': 'log2', 'n_e..."
3,xgb,0.821752,"{'colsample_bytree': 0.7, 'learning_rate': 0.1..."
4,bagging,0.825378,"{'estimator': DecisionTreeClassifier(), 'max_f..."
